In [2]:
import veryfi
import os
import json

# Configuración de credenciales de Veryfi
client_id = 'vrfhjAs5kPIee2Yvy7F0osVV5MX6zXKFTriJyLS'
client_secret = 'NJoXZcE25MBxv97merROYW5CvYBJLmNwrGDoYEWzv970T0fyeE2pxAXlP93huX46Bf1NbgM6jGzVb3LzSrTWZYDEGcyaruHwfWAr75EfA87rcoo29NLYzWRWJdogbc1N'
username = 'juanpamon.13'
api_key = 'f604ee697fd73c0ba1bb99a536bcde0b'

client = veryfi.Client(client_id, client_secret, username, api_key)

# Directorios
input_folder = "Documents"
output_folder = "outputs"

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Procesar todos los archivos PDF en la carpeta Documents
for file_name in os.listdir(input_folder):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(input_folder, file_name)
        print(f"Procesando: {file_name}")

        # Extraer datos con Veryfi
        json_result = client.process_document(file_path)

        # Guardar el resultado en la carpeta outputs
        output_file = os.path.join(output_folder, f"{file_name}.json")
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(json_result, f, ensure_ascii=False, indent=4)

        print(f"Resultado guardado en: {output_file}")

print("Procesamiento completado.")



Procesando: synth-switch_v5-14.pdf
Resultado guardado en: outputs\synth-switch_v5-14.pdf.json
Procesando: synth-switch_v5-4.pdf
Resultado guardado en: outputs\synth-switch_v5-4.pdf.json
Procesando: synth-switch_v5-68.pdf
Resultado guardado en: outputs\synth-switch_v5-68.pdf.json
Procesando: synth-switch_v5-7.pdf
Resultado guardado en: outputs\synth-switch_v5-7.pdf.json
Procesando: synth-switch_v5-79.pdf
Resultado guardado en: outputs\synth-switch_v5-79.pdf.json
Procesamiento completado.


In [3]:
ocr_text = json_result.get("ocr_text", "")

In [4]:
ocr_text

'\tInvoice\n\tPage 1 of 3\nswitch\tDallas, TX 75267-4592\nPO Box 674592\n\n\tInvoice Date Due Date\tInvoice No.\n\t01/12/24\t11/24/24\t9230090\n\nIncentX\n2929 Main St\nSanta Monica, CA 90405-5315\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of August\nW-300387\t\t\t\t\tPO-TMQ7-4458\nDescription\t\t\t\tQuantity\tRate\tAmount\nInstallation of Cross Connect | 42 Gbps Fiber to pK2A19iL\t\t\t\t568.46\t8,013.00\t4,555,069.98\nTransport | 23 Gbps Fiber to oX0U2WYCkp (PVJTZ3D1) (02/2024)\t\t\t166.06\t1,728.11\t286,969.95\nTransport | 88 Gbps Fiber to don0g Dz (0EYML836) (01/2024)\t\t\t15.25\t3,714.52\t56,646.43\nCarrier Taxes for Transport | 75 Gbps Wavelength Diverse between Hillsboro, OR 12345\t844.80\t3,596.97\t3,038,720.26\nand LasVegas, OR 12345 (ulmuh) (9HZIFE0Q) (07/2023 Taxes) (07/2023)\nInstallation of Cross Connect | 77 Gbps Fiber to Mk8Fdp\t\t\t\t4,361.02\t4,321.33\t18,845,406.56\nTransport | 671 Gbps Fiber to YdxQR4xvZ5 (XHXSNF8V) (05/2023)\t\t\t1,484.48\t6,007.97\t8,918,

In [29]:
import re
import json

def extract_invoice_data(ocr_text):
    """
    Extrae la información clave de la factura desde el texto OCR.
    """

    invoice_data = {}

    # Vendor Name
    vendor_match = re.search(r'Please make payments to:\s*(.+)', ocr_text)
    invoice_data["vendor_name"] = vendor_match.group(1).strip() if vendor_match else "Unknown"

    # Vendor Address (Fijo)
    vendor_address_match = re.search(r"^\s*Page \d+ of \d+\n\s*\S+\s*(.*?)\n\n", ocr_text, re.DOTALL | re.MULTILINE)
    invoice_data["vendor_address"] = vendor_address_match.group(1).strip() if vendor_address_match else "Unknown"

    # Invoice Number y Bill To Name
    invoice_match = re.search(r'Invoice No\.\n\t(\d{2}/\d{2}/\d{2})\t(\d{2}/\d{2}/\d{2})\t([\d]+)\n\n(.+)', ocr_text)
    if invoice_match:
        invoice_data["date"] = invoice_match.group(1)  # Primera fecha después de Invoice No.
        invoice_data["invoice_number"] = invoice_match.group(3)  # Número de factura
        invoice_data["bill_to_name"] = invoice_match.group(4).strip()  # Nombre después de Invoice No.
    else:
        invoice_data["date"] = "Unknown"
        invoice_data["invoice_number"] = "Unknown"
        invoice_data["bill_to_name"] = "Unknown"

    # Extraer Items
    items = []
    item_pattern = re.findall(
        r'([^\n]+?)\s+\(?(\d+)?\)?\s+([\d,]+\.\d+)\s+([\d,]+\.\d+)\s+([\d,]+\.\d+)',
        ocr_text
    )

    for item in item_pattern:
        description, sku, quantity, price, total = item
        items.append({
            "sku": sku if sku else "Unknown",
            "description": description.strip(),
            "quantity": float(quantity.replace(',', '')),
            "tax_rate": None,  # No está presente en la factura
            "price": float(price.replace(',', '')),
            "total": float(total.replace(',', ''))
        })

    invoice_data["items"] = items

    return invoice_data

# OCR Text de prueba
#ocr_text = '''(Pega aquí tu texto OCR)'''

# Extraer datos
invoice_info = extract_invoice_data(ocr_text)

# Guardar en JSON
with open("outputs/invoice_data.json", "w", encoding="utf-8") as f:
    json.dump(invoice_info, f, ensure_ascii=False, indent=4)

print(json.dumps(invoice_info, indent=4, ensure_ascii=False))

{
    "vendor_name": "Switch, Ltd.",
    "vendor_address": "Dallas, TX 75267-4592\nPO Box 674592",
    "date": "01/12/24",
    "invoice_number": "9230090",
    "bill_to_name": "IncentX",
    "items": [
        {
            "sku": "Unknown",
            "description": "Installation of Cross Connect | 42 Gbps Fiber to pK2A19iL",
            "quantity": 568.46,
            "tax_rate": null,
            "price": 8013.0,
            "total": 4555069.98
        },
        {
            "sku": "Unknown",
            "description": "Transport | 23 Gbps Fiber to oX0U2WYCkp (PVJTZ3D1) (02/2024)",
            "quantity": 166.06,
            "tax_rate": null,
            "price": 1728.11,
            "total": 286969.95
        },
        {
            "sku": "Unknown",
            "description": "Transport | 88 Gbps Fiber to don0g Dz (0EYML836) (01/2024)",
            "quantity": 15.25,
            "tax_rate": null,
            "price": 3714.52,
            "total": 56646.43
        },
        {


In [21]:
import re

text = """
    \tInvoice\n\tPage 1 of 3\nswitch\tDallas, TX 75267-4592\nPO Box 674592\n\n\tInvoice Date Due Date\tInvoice No.\n\t01/12/24\t11/24/24\t9230090\n\nIncentX\n2929 Main St\nSanta Monica, CA 90405-5315\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of August\nW-300387\t\t\t\t\tPO-TMQ7-4458\nDescription\t\t\t\tQuantity\tRate\tAmount\nInstallation of Cross Connect
"""

a = re.search(r"^\s*Page 1 of 3\n(.*?)\n\n", ocr_text, re.DOTALL | re.MULTILINE)

if a:  # Comprobar si 'a' no es None
    vendor_address = a.group(1).strip()
    print(vendor_address)
else:
    print("No se encontró la dirección del vendedor.")

switch	Dallas, TX 75267-4592
PO Box 674592
